In [5]:

import os
import cv2
import numpy as np
import math
from tqdm import tqdm
import pandas as pd

from face_adjustments_module import FaceAdjuster
from face_mesh_module import FaceMeshDetector
from definitions import *

In [15]:
def analyseFace(image, extractor):
    # print("analyseFace")
    row, col = image.shape[:2]
    img_new_width = initial_image_width
    rt = img_new_width/col
    image = cv2.resize(image, (img_new_width, int(row*rt)))

    lms = extractor.findFaceMesh(image.copy())
    if(not lms):
        print("\nNo faces")
        return [], "No Faces detected"
    lms = lms[0]
    # --------------------------------------------------------------------------
    adjuster = FaceAdjuster(image.copy(), lms)
    _, succeed = adjuster.alignEyes()
    if not succeed:
        return [], adjuster.error
    _, succeed = adjuster.alignFace()
    if not succeed:
        return [], adjuster.error
    _, succeed = adjuster.faceCrop()
    if not succeed:
        return [], adjuster.error
    _, succeed = adjuster.alignFace()
    if not succeed:
        return [], adjuster.error
    finalImage, succeed = adjuster.fixImageSizeWitBorders()
    if not succeed:
        return [], adjuster.error

    nlms = adjuster.getLms()

    return nlms, finalImage, None

In [17]:
def process_video(path = ""):
    
    camera = cv2.VideoCapture( str(path))
    vfps = (camera.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    name = path.split('/')
    name = name[len(name)-1].split('.')
    name = name[0]
    try:
        os.mkdir('./vds/prc/'+name )
    except:
        print("Diretorio ja existe")
    path = './vds/prc/'+name +"/"
    name =  './vds/prc/'+name + '/video.avi'
    vLength = int(camera.get(cv2.CAP_PROP_FRAME_COUNT))
    #file = open(path+"data.pickle", 'rb')
    (h,w) = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(name, fourcc,int(vfps),(h,w))
    
    past_positions = {
        "left": [],
        "right": []
    }
    landmarks_extractor = FaceMeshDetector()
    for i in tqdm(range (0,vLength+1)):
        data = {
            "left_eye": {
                "raw": None,
                "blurred": None,
                "binary": None,
                "histograms":{
                    "rows": None,
                    "columns": None
                },
                "x": 0,
                "y":0
            },
            "right_eye": {
                "raw": None,
                "blurred": None,
                "binary": None,
                "histograms":{
                    "rows": None,
                    "columns": None
                },
                "x": 0,
                "y":0
            }
        }
        ret, frame = camera.read()
        if ret: 
            clear_image = frame.copy()
            try:
                lms, fimage, err = analyseFace(
                    cv2.imread(path), landmarks_extractor)
            except Exception as exception:
                err = type(exception).__name__
                print()
                print(err)
                break
            cv2.imshow("adjusted",fimage)
            

In [18]:
process_video(path = "./vds/raw/"+"rodrigo.mp4")

Diretorio ja existe


  0%|          | 0/919 [00:00<?, ?it/s]


AttributeError
